# Building an image classifier using the sequential API

Following step-by-step procedure in Chapter 10 of Aurélien Géron's book _Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow_.

In [1]:
from pathlib import Path, PurePath
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import h5py
%matplotlib widget

In [2]:
import tensorflow
print(f'TensorFlow version: {tensorflow.__version__}')
from tensorflow import keras
print(f'Keras version: {keras.__version__}')

TensorFlow version: 2.0.0
Keras version: 2.2.4-tf


In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

We will be using the _Fashion MNIST data set_, which has fashion image items instead of numbers.

In [4]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [5]:
print(f'X_train_full.shape: {X_train_full.shape}')
print(f'X_train_full.dtype: {X_train_full.dtype}')

X_train_full.shape: (60000, 28, 28)
X_train_full.dtype: uint8


In [6]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

## Build model with sequential API

In [8]:
model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]), \
                                 keras.layers.Dense(300, activation="relu"), \
                                 keras.layers.Dense(100, activation="relu"), \
                                 keras.layers.Dense(10, activation="softmax")])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.layers

In [11]:
hidden1 = model.layers[1]
hidden1.name

'dense'

In [12]:
model.get_layer('dense_2') is hidden1

False

In [13]:
weights, biases = hidden1.get_weights()

### Compiling the model

In [14]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [15]:
t0 = time()
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))
print(f'Time elapsed: {time() - t0:.4f} sec')

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 8s 150us/sample - loss: 0.7047 - accuracy: 0.7670 - val_loss: 0.5093 - val_accuracy: 0.8230
Epoch 2/30
55000/55000 [==============================] - 7s 120us/sample - loss: 0.4863 - accuracy: 0.8309 - val_loss: 0.4569 - val_accuracy: 0.8448
Epoch 3/30
55000/55000 [==============================] - 7s 127us/sample - loss: 0.4412 - accuracy: 0.8455 - val_loss: 0.4127 - val_accuracy: 0.8590
Epoch 4/30
55000/55000 [==============================] - 7s 130us/sample - loss: 0.4133 - accuracy: 0.8564 - val_loss: 0.4038 - val_accuracy: 0.8564
Epoch 5/30
55000/55000 [==============================] - 7s 131us/sample - loss: 0.3937 - accuracy: 0.8614 - val_loss: 0.3747 - val_accuracy: 0.8704
Epoch 6/30
55000/55000 [==============================] - 7s 136us/sample - loss: 0.3775 - accuracy: 0.8666 - val_loss: 0.4519 - val_accuracy: 0.8286
Epoch 7/30
55000/55000 [===========================

In [16]:
history.params

{'batch_size': 32,
 'epochs': 30,
 'steps': 1719,
 'samples': 55000,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'accuracy', 'val_loss', 'val_accuracy']}

In [17]:
df = pd.DataFrame(history.history)
df.tail(10)

,loss,accuracy,val_loss,val_accuracy
20,0.263550,0.904473,0.315249,0.8878
21,0.258811,0.906782,0.310559,0.8878
22,0.254937,0.907855,0.303981,0.8888
23,0.249726,0.908964,0.303086,0.8920
24,0.245319,0.911218,0.298303,0.8924
25,0.242227,0.911800,0.302030,0.8942
26,0.237853,0.913182,0.296242,0.8918
27,0.233487,0.915745,0.306152,0.8860
28,0.230188,0.918236,0.318109,0.8882
29,0.227200,0.917527,0.296415,0.8926


In [19]:
df.plot(figsize=(10, 6))
plt.grid(True)
plt.gca().set_ylim(0, 1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …